<a href="https://colab.research.google.com/github/aebongbing/ESAA/blob/main/ESAA_0509_%ED%95%84%EC%82%AC_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 09. 추천 시스템

1. 콘테츠 기반 필터링 추천 시스템
- 사용자가 특정한 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식

 ex) 사용자가 특정 영화에 높은 평점을 줬다면 그 영화의 장르, 출연 배우, 감독, 영화 키워드 등의 콘텐츠와 유사한 다른 영화를 추천해주는 방식

2. 협업 필터링

   - 사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식만을 기반으로 추천을 수행하는 것

   - 주요 목표: 사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가하는 것

   - 최근접 이웃 방식과 잠재 요인 방식으로 나뉘며 두 방식 모두 사용자-아이템 평점 행렬 데이터에만 의지해 추천을 수행함, 사용자-아이템 평점 행렬에서 행은 개별 사용자, 열은 개별 아이템으로 구성되며 사용자 아이디 행, 아이템 아이디 열 위치에 해당하는 값이 평점을 나타내는 형태가 되어야함, 일반적으로 이러한 사용자-아이템 평점 행렬은 많은 아이템을 열로 가지는 다차원 행렬이며, 사용자가 아이템에 대한 평점을 매기는 경우가 많지 않기 때문에 희소 행렬 특성을 가지고 있음

   - 최근접 이웃 협업 필터링
    
     - 사용자 기반: 특정 사용자와 유사한 다른 사용자를 TOP-N으로 선정해 이 TOP-N 사용자가 좋아하는 아이템을 추천하는 방식

     - 아이템 기반:사용자들이 그 아이템을 좋아하는지/ 싫어하는지의 평가 척도가 유사한 아이템을 추천하는 기준이 되는 알고리

     - 일반적으로 아이템 기반 협업 필터링이 정확도가 더 높음

   - 잠재 요인 협업 필터링

     - 대규모 다차원 행렬을 차원 감소 기법으로 분해하는 과정(행렬 분해)에서 잠재적 요인을 추출해 추천 예측을 할 수 있게 하는 기법

     - 사용자-아이템 평점 행렬 데이터만을  이용함

     - 알고리즘 골자: 다차원 희소 행렬인 사용자-아이템 행렬 데이터를 저차원 밀집 행렬의 사용자-잠재요인 행렬과 잠재요인-아이템 행렬로 분해할 수 있으며, 이렇게 분해된 두 행렬의 내적을 통해 새로운 예측 사용자-아이템 평점 행렬 데이터를 만들어서 사용자가 아직 평점을 부여하지 않는 아이템에 대한 예측 평점을 생성하는 것

     

   

In [3]:
#SGD를 이용해 행렬 분해를 수행하는 예제
#분해하려는 원본 행렬 R을 P와 Q로 분해한 뒤에 다시 P와 Q.T의 내적으로 예측 행렬을 만드는 예제
#원본 행렬 R을 미정인 널 값을 포함해 생성하고 분해 행렬 P와 Q는 정규 분포를 가진 랜덤 값으로 초기화
#잠재요인 차원은 3으로 설정

import numpy as np

R=np.array([[4, np.NaN, np.NaN, 2, np.NaN],
            [np.NaN, 5, np.NaN, 3, 1],
            [np.NaN, np.NaN, 3, 4, 4],
            [5, 2, 1, 2,np.NaN]])
num_users, num_items=R.shape
k=3

np.random.seed(1)
P=np.random.normal(scale=1./k,size=(num_users, k))
Q=np.random.normal(scale=1./k,size=(num_items, k))

In [5]:
from sklearn.metrics import mean_squared_error
def get_rmse(R,P,Q,non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P,Q.T)

    #실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [9]:
# R >0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros=[(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]

steps=1000
learning_rate=0.01
r_lambda=0.01

#SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
  for i, j , r in non_zeros:
    #실제 값과 예측 값의 차이인 오류 값 구함
    eij=r-np.dot(P[i,:],Q[j,:].T)
    #Regularization을 반영한 SGD 업데이트 공식 적용
    P[i,:]=P[i,:]+learning_rate*(eij*Q[j,:]-r_lambda*P[i,:])
    Q[j,:]=Q[j,:]+learning_rate*(eij*P[i,:]-r_lambda*Q[j,:])
    rmse=get_rmse(R, P, Q, non_zeros)
    if (step % 50)==0:
      print("### iteration step:",step,"rmse:",rmse)

### iteration step: 0 rmse: 3.2475255402584495
### iteration step: 0 rmse: 3.2465908465596782
### iteration step: 0 rmse: 3.240001170538425
### iteration step: 0 rmse: 3.2381485790985485
### iteration step: 0 rmse: 3.2383355624029324
### iteration step: 0 rmse: 3.237066663018253
### iteration step: 0 rmse: 3.235237278782524
### iteration step: 0 rmse: 3.2334055187092163
### iteration step: 0 rmse: 3.228120562324444
### iteration step: 0 rmse: 3.22631720731668
### iteration step: 0 rmse: 3.226425162809075
### iteration step: 0 rmse: 3.225031693566006
### iteration step: 50 rmse: 0.492114962541484
### iteration step: 50 rmse: 0.49188671936595735
### iteration step: 50 rmse: 0.4908767627014268
### iteration step: 50 rmse: 0.490706148990846
### iteration step: 50 rmse: 0.49080798389226865
### iteration step: 50 rmse: 0.4901647116343035
### iteration step: 50 rmse: 0.49024508668575006
### iteration step: 50 rmse: 0.489756757805949
### iteration step: 50 rmse: 0.48218311207525344
### iterati

In [10]:
#P와 Q함수를 P*Q.T로 예측 행렬을 만들어서 출력
pred_matrix=np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.99  0.804 1.334 2.002 1.714]
 [6.67  4.978 0.962 2.98  1.003]
 [6.843 0.408 2.987 3.977 3.986]
 [4.968 2.005 1.007 2.018 1.158]]
